In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#importing all the required libraries 

In [2]:
# Hyper Parameters #

learning_rate = 1e-3
num_epochs = 20
batch_size = 256
num_class = 10

In [3]:
class SimpleCNN(nn.Module):
    """
    defining simple CNN with 2 convolution layers, 2 pooling layers and 2 fully-connected layers
    """
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Network's Layers
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.max1 = nn.MaxPool2d(2, stride=2)
        self.max2 = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, num_class)
        # Dropout
        self.dropout1 = nn.Dropout(0.45)
        self.dropout2 = nn.Dropout(0.35)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(32)
        self.batch2 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = self.conv1(x)                   # Convolution layer
        x = F.relu(self.batch1(x))          # Batch Normalization + ReLU
        x = self.max1(x)                    # Max-pool layer
        x = self.dropout1(x)                # Dropout

        x = self.conv2(x)                   # Convolution layer
        x = F.relu(self.batch2(x))          # Batch Normalization + ReLU
        x = self.max2(x)                    # Max-pool layer
        x = self.dropout2(x)                # Dropout

        x = x.reshape(-1, 1024)             # Flatten
        x = F.relu(self.fc1(x))             # Fully-connected layer + ReLU
        x = F.softmax(self.fc2(x), dim=0)   # Fully-connected layer + Softmax
        return x

In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device, '\n')

net = SimpleCNN()
net.to(device)


Using cpu 



SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (max1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (max2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (dropout1): Dropout(p=0.45, inplace=False)
  (dropout2): Dropout(p=0.35, inplace=False)
  (batch1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [5]:
# Data #
"""
MNIST datasets from torchvision.datasets
"""
train_dataset = datasets.MNIST(root='./data', train=True,
                               download=True, transform=ToTensor())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False,
                              download=True, transform=ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)>



100%|████████████████████████████| 9912422/9912422 [00:01<00:00, 8249928.22it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)>



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 350426.24it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)>



100%|████████████████████████████| 1648877/1648877 [00:00<00:00, 3149205.47it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)>



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 1130729.39it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# Loss & Optimization #
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [7]:
# Train #
for epoch in range(num_epochs):
    correct_train = 0
    total_train = 0

    net.train()  # training mode
    print('epoch #{}' .format(epoch+1))
    for i, (inputs, labels) in enumerate(train_loader):
        inputs.to(device), labels.to(device)

        # set gradient's parameter to zero
        optimizer.zero_grad()

        # forward pass & back propagation
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # accuracy calculation
        _, predicted = torch.max(outputs.data, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    print('Train Accuracy: %0.3f' % ((100 * correct_train) / total_train))

epoch #1
Train Accuracy: 80.617
epoch #2
Train Accuracy: 85.842
epoch #3
Train Accuracy: 91.422
epoch #4
Train Accuracy: 93.188
epoch #5
Train Accuracy: 93.933
epoch #6
Train Accuracy: 94.500
epoch #7
Train Accuracy: 94.813
epoch #8
Train Accuracy: 95.137
epoch #9
Train Accuracy: 95.412
epoch #10
Train Accuracy: 95.620
epoch #11
Train Accuracy: 95.735
epoch #12
Train Accuracy: 96.008
epoch #13
Train Accuracy: 96.083
epoch #14
Train Accuracy: 96.003
epoch #15
Train Accuracy: 96.143
epoch #16
Train Accuracy: 95.938
epoch #17
Train Accuracy: 95.860
epoch #18
Train Accuracy: 96.292
epoch #19
Train Accuracy: 96.225
epoch #20
Train Accuracy: 96.407


In [8]:
# Test #
net.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_loader):
        inputs.to(device), labels.to(device)

        outputs = net(inputs)
        loss = criterion(outputs, labels)
    
        # test accuracy calculation
        _, predicted = torch.max(outputs.data, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

print('\nTest Accuracy: %0.3f' % ((100 * correct_test) / total_test))


Test Accuracy: 97.500
